In [1]:
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Pride and Prejudice.txt to Pride and Prejudice (1).txt


In [3]:
file = open("Pride and Prejudice.txt", 'r', encoding = 'utf8')

# store file in list
lines = []
for i in file:
  lines.append(i)

# convert list to string
data = ""
for i in lines:
  data += i
  data += ' '

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r','').replace('\ufeff','').replace('“','').replace('”','')

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of Pride and Prejudice, by Jane Austen This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using th'

In [4]:
len(data)

698418

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 176, 158, 916, 3, 321, 4, 1171, 30, 72, 2534, 41, 916, 23, 21]

In [6]:
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

7030


In [7]:
sequences = []

for i in range(3, len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)

print('The length of sequences are: ', len(sequences))
sequences = np.array(sequences)
sequences[:10]

The length of sequences are:  125306


array([[   1,  176,  158,  916],
       [ 176,  158,  916,    3],
       [ 158,  916,    3,  321],
       [ 916,    3,  321,    4],
       [   3,  321,    4, 1171],
       [ 321,    4, 1171,   30],
       [   4, 1171,   30,   72],
       [1171,   30,   72, 2534],
       [  30,   72, 2534,   41],
       [  72, 2534,   41,  916]])

In [8]:
X = []
y = []

for i in sequences:
  X.append(i[0:3])
  y.append(i[3])

X = np.array(X)
y = np.array(y)

In [18]:
print('Data: ', X[:10])
print('Response: ', y[:10])

Data:  [[   1  176  158]
 [ 176  158  916]
 [ 158  916    3]
 [ 916    3  321]
 [   3  321    4]
 [ 321    4 1171]
 [   4 1171   30]
 [1171   30   72]
 [  30   72 2534]
 [  72 2534   41]]
Response:  [ 916    3  321    4 1171   30   72 2534   41  916]


In [9]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             70300     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 7030)              7037030   
                                                                 
Total params: 20,156,330
Trainable params: 20,156,330
Non-trainable params: 0
_________________________________________________________________


In [12]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only = True)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
1957/1958 [============================>.] - ETA: 0s - loss: 6.2429
Epoch 1: loss improved from inf to 6.24281, saving model to next_words.h5
1958/1958 [==============================] - 38s 15ms/step - loss: 6.2428
Epoch 2/70
1956/1958 [============================>.] - ETA: 0s - loss: 5.6147
Epoch 2: loss improved from 6.24281 to 5.61426, saving model to next_words.h5
1958/1958 [==============================] - 31s 16ms/step - loss: 5.6143
Epoch 3/70
1956/1958 [============================>.] - ETA: 0s - loss: 5.2805
Epoch 3: loss improved from 5.61426 to 5.28063, saving model to next_words.h5
1958/1958 [==============================] - 30s 15ms/step - loss: 5.2806
Epoch 4/70
1958/1958 [==============================] - ETA: 0s - loss: 5.0585
Epoch 4: loss improved from 5.28063 to 5.05851, saving model to next_words.h5
1958/1958 [==============================] - 30s 15ms/step - loss: 5.0585
Epoch 5/70
1955/1958 [============================>.] - ETA: 0s - loss: 4.8650
E

In [13]:
from keras.models import load_model

model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""

  for key, value in tokenizer.word_index.items():
    if value == preds:
      predicted_word = key
      break

  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
    print("Execution completed...")
    break
  
  else:
    try:
      text = text.split(" ")
      text = text[-3:]
      print(text)

      Predict_Next_Words(model, tokenizer, text)
    
    except Exception as e:
      print("Error occurred: ", e)
      continue

Enter your line:  It is a truth universally
['a', 'truth', 'universally']
1/1 [==============================] - 1s 974ms/step
acknowledged
Enter your line: taken by a young man of large
['man', 'of', 'large']
1/1 [==============================] - 0s 18ms/step
fortune
